In [1]:
import numpy as np
import glob, os
from keras import layers as ly
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from keras.preprocessing import sequence
from sklearn.model_selection import train_test_split
import keras.backend.tensorflow_backend as K

Using TensorFlow backend.


In [2]:
dir = './dfs_sequence/*'

In [3]:
# file read
all_data = []
sequence_length = []
for file in sorted(glob.glob(dir)):
    datasets = []
    for f in open(file, 'r'):
        f = f.replace(']', '').replace('[', '').replace('\n','')
        (u, v, w) = f.split(',')
        datasets.append([int(u), int(v), float(w)])
    sequence_length.append(len(datasets))
    all_data.append(datasets)
#all_data = np.array(all_data)
all_data = np.array([np.array(arr) for arr in all_data])

In [4]:
x_train, x_test = train_test_split(all_data, test_size = 0.2)
x_test, x_val = train_test_split(x_test, test_size = 0.66)

In [5]:
n_features = 3
batch_size = 32
epochs = 100

In [6]:
len(x_train)

756

In [ ]:
from keras.optimizers import Adam
from keras.layers import Input, LSTM, RepeatVector, Lambda
from keras.models import Model

def repeat_vector(args):
    layer_to_repeat = args[0]
    sequence_layer = args[1]
    return RepeatVector(K.shape(sequence_layer)[1])(layer_to_repeat)
    

inputs = Input(shape=(None, 3))
encoded = LSTM(128, return_sequences=True)(inputs)  #activation 안적으면 tanh
encoded = LSTM(64)(encoded)

decoded = Lambda(repeat_vector, output_shape=(None, 64)) ([encoded, inputs]) # inputs의 shape[1] 만큼 encoded 를 반복 생성

decoded = LSTM(64, return_sequences=True)(decoded)
decoded = LSTM(128, return_sequences=True)(decoded)
decoded = TimeDistributed(Dense(3))(decoded)
encoder = Model(inputs, encoded)

lstm_autoencoder = Model(inputs, decoded)
lstm_autoencoder.compile(loss='mean_squared_error', optimizer=Adam(lr=0.001))

def train_generator(x_train):
    idx = 0
    while True:
        yield np.array([x_train[idx]]), np.array([x_train[idx]])
        idx +=1
        if idx >= len(x_train):
            idx = 0
lstm_autoencoder.fit_generator(train_generator(x_train), epochs=500, steps_per_epoch=756, verbose=1)#, validation_data=val_generator(x_val))

''' result

'''

Epoch 1/500
756/756 [==============================] - 9s 12ms/step - loss: 47.6620
Epoch 2/500
756/756 [==============================] - 7s 9ms/step - loss: 36.7579
Epoch 3/500
756/756 [==============================] - 7s 10ms/step - loss: 28.2871
Epoch 4/500
756/756 [==============================] - 7s 10ms/step - loss: 25.0947
Epoch 5/500
756/756 [==============================] - 7s 9ms/step - loss: 23.2867
Epoch 6/500
756/756 [==============================] - 9s 11ms/step - loss: 22.1029
Epoch 7/500
756/756 [==============================] - 8s 10ms/step - loss: 20.4334
Epoch 8/500
756/756 [==============================] - 8s 10ms/step - loss: 19.6250
Epoch 9/500
756/756 [==============================] - 8s 11ms/step - loss: 18.3419
Epoch 10/500
756/756 [==============================] - 7s 9ms/step - loss: 18.0927
Epoch 11/500
756/756 [==============================] - 8s 10ms/step - loss: 16.5675
Epoch 12/500
756/756 [==============================] - 7s 9ms/step - loss: 1

In [ ]:
encoder = Model(inputs, encoded)

In [ ]:
lstm_autoencoder.summary()

In [ ]:
encoder.summary()

In [ ]:
lstm_autoencoder.summary()

In [42]:
lstm_autoencoder.predict(x_test[0].reshape(1, x_test[0].shape[0], x_test[0].shape[1]))

array([[[ 8.931337 ,  7.4579573, 31.943764 ],
        [ 8.431695 ,  7.737891 ,  8.375352 ],
        [ 7.3948007,  5.0614414, 13.785835 ],
        [ 7.925072 ,  5.933628 , 14.485416 ],
        [ 2.0842404,  1.931783 ,  8.705457 ],
        [ 2.2869372,  3.9760273,  7.5900183],
        [ 7.5529623,  5.3252344, 26.848215 ],
        [ 2.7739074,  3.8996158,  8.020057 ],
        [ 1.5346651,  4.882212 , 11.469937 ]]], dtype=float32)

In [76]:
test = x_test[0].reshape(1, 9, 3)
latent_vector = []
for x in x_test:
    x = x.reshape(1, x.shape[0], x.shape[1])
    latent_vector.append(encoder.predict(x)[0])

In [144]:
def printMatrix(matrix):
    length = int(max(max(matrix[:, 0]), max(matrix[:, 1])))
    adj = [[0 for i in range(length)] for i in range(length)]
    
    for m in matrix:
        (i, j, w) = m
        i = int(i) -1
        j = int(j) -1
        adj[i][j] = adj[j][i] = w
    for i in range(length):
        for j in range(length):
            print(str(adj[i][j]), end=',')
        print()
    print("===============")

In [213]:
# sequence to matrix
for index, x in enumerate(x_test):
    print("#", str(index))
    printMatrix(x)
    

# 0
0,10.16,7.92,9.56,0,0,0,0,0,
10.16,0,0,0,0,0,0,0,0,
7.92,0,0,0,8.17,18.05,0,0,0,
9.56,0,0,0,0,0,20.05,0,0,
0,0,8.17,0,0,0,0,0,0,
0,0,18.05,0,0,0,0,32.09,0,
0,0,0,20.05,0,0,0,0,20.27,
0,0,0,0,0,32.09,0,0,9.01,
0,0,0,0,0,0,20.27,9.01,0,
# 1
0,8.79,11.79,0,0,0,
8.79,0,0,10.06,6.86,0,
11.79,0,0,0,0,7.86,
0,10.06,0,0,0,0,
0,6.86,0,0,0,0,
0,0,7.86,0,0,0,
# 2
0,3.3,6.3,0,0,
3.3,0,0,12.72,0,
6.3,0,0,0,23.87,
0,12.72,0,0,41.95,
0,0,23.87,41.95,0,
# 3
0,5.7,7.22,0,0,
5.7,0,0,12.22,0,
7.22,0,0,0,23.54,
0,12.22,0,0,42.39,
0,0,23.54,42.39,0,
# 4
0,12.52,17.52,28.77,0,0,0,0,0,
12.52,0,0,0,0,0,0,0,0,
17.52,0,0,0,2.53,4.53,0,0,0,
28.77,0,0,0,0,0,10.86,0,0,
0,0,2.53,0,0,0,0,0,0,
0,0,4.53,0,0,0,0,9.01,0,
0,0,0,10.86,0,0,0,0,10.01,
0,0,0,0,0,9.01,0,0,20.01,
0,0,0,0,0,0,10.01,20.01,0,
# 5
0,4.27,9.29,0,0,
4.27,0,0,12.29,0,
9.29,0,0,0,17.29,
0,12.29,0,0,3.95,
0,0,17.29,3.95,0,
# 6
0,10.16,7.92,9.56,0,0,0,0,0,
10.16,0,0,0,0,0,0,0,0,
7.92,0,0,0,8.17,18.05,0,0,0,
9.56,0,0,0,0,0,20.05,0,0,
0,0,8.17,0,0,0,0

In [171]:
standard = latent_vector[0]

In [223]:
import copy
dist = []
left = 100000
right = -10000
for index, v in enumerate(latent_vector[1:]):
    d = round(euclidean_distance(standard, v), 8)
    dist.append(d)
    if d > right:
        right= d
    if left > d:
        left = d
dist1 = copy.deepcopy(dist)
dist1.sort()
#print(dist, dist1)
for index, d in enumerate(dist1):
    print( (index+1, dist.index(d)+1), end='  ')

(1, 2)  (2, 38)  (3, 53)  (4, 5)  (5, 15)  (6, 44)  (7, 48)  (8, 21)  (9, 12)  (10, 43)  (11, 9)  (12, 18)  (13, 31)  (14, 59)  (15, 58)  (16, 4)  (17, 41)  (18, 27)  (19, 29)  (20, 33)  (21, 6)  (22, 3)  (23, 51)  (24, 35)  (25, 54)  (26, 55)  (27, 26)  (28, 62)  (29, 14)  (30, 46)  (31, 11)  (32, 45)  (33, 7)  (34, 50)  (35, 57)  (36, 8)  (37, 63)  (38, 60)  (39, 24)  (40, 52)  (41, 20)  (42, 13)  (43, 37)  (44, 23)  (45, 10)  (46, 1)  (47, 61)  (48, 42)  (49, 56)  (50, 34)  (51, 16)  (52, 32)  (53, 40)  (54, 39)  (55, 19)  (56, 17)  (57, 22)  (58, 49)  (59, 30)  (60, 36)  (61, 47)  (62, 28)  (63, 25)  

In [168]:
def euclidean_distance(a, b):
    return np.sqrt(np.sum(a-b) ** 2)

In [221]:
len(x_test)

64